In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torchvision
from sklearn.metrics import confusion_matrix  
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import VisionDataset
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import os.path
import sys
from torch.backends import cudnn
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

from datasets import Cityscapes
from clients import *
from servers import *
from utils import *
from models import *


In [3]:
#DEVICE = 'cuda' # 'cuda' or 'cpu'
DEVICE = 'cuda'
NUM_CLASSES = 19

BATCH_SIZE = 16     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 5*1e-3           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  

NUM_EPOCHS = 25     
STEP_SIZE = 20       #20 How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

In [4]:
#transformations
size = (256, 512) #512,1024
scales=(0.25, 2.)
cropsize=(512, 1024)
eval_crop=(1024, 1024)
eval_scales=(0.5, 0.75, 1.0, 1.25, 1.5, 1.75)
test_transformations = [RandomResizedCrop(eval_scales, eval_crop)]
train_transformations = [RandomResizedCrop(scales,cropsize),RandomHorizontalFlip(),ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4)]
test_transformations = Compose(test_transformations)
train_transformations = Compose(train_transformations)

In [5]:

clients = []
datasets = []
styles = []
with open("/content/drive/MyDrive/data/data/Cityscapes/train_A_I.txt") as lines:
  curr_id = 0
  paths = []
  for line in lines:
    id_client,path = line.split(" ")
    if curr_id == int(id_client):
      paths.append(path)
    else:
      print(f"finished Client ID:[{curr_id}]: len: {len(paths)}")
      dataset = DatasetClient("/content/drive/MyDrive/data/data/Cityscapes",paths,train_transformations)
      datasets.append(dataset)
      client = Client(curr_id,dataset)
      clients.append(client)

      paths = []
      paths.append(path)
      curr_id = int(id_client)  

  dataset = DatasetClient("/content/drive/MyDrive/data/data/Cityscapes",paths,train_transformations)
  datasets.append(dataset)
  client = Client(curr_id,dataset)
  clients.append(client)
  client.compute_avg_style()
  styles.append(deepcopy(client.style_detector))
  
#print(len(clients), f"last: {curr_id}")
i,_ = datasets[0].__getitem__(0)
print(i.shape)

finished Client ID:[0]: len: 20
finished Client ID:[1]: len: 20
finished Client ID:[2]: len: 20
finished Client ID:[3]: len: 20
finished Client ID:[4]: len: 20
finished Client ID:[5]: len: 20
finished Client ID:[6]: len: 20
finished Client ID:[7]: len: 20
finished Client ID:[8]: len: 20
finished Client ID:[9]: len: 20
finished Client ID:[10]: len: 20
finished Client ID:[11]: len: 20
finished Client ID:[12]: len: 20
finished Client ID:[13]: len: 20
finished Client ID:[14]: len: 20
finished Client ID:[15]: len: 20
finished Client ID:[16]: len: 20
finished Client ID:[17]: len: 20
finished Client ID:[18]: len: 20
finished Client ID:[19]: len: 20
finished Client ID:[20]: len: 20
finished Client ID:[21]: len: 20
finished Client ID:[22]: len: 20
finished Client ID:[23]: len: 20
finished Client ID:[24]: len: 20
finished Client ID:[25]: len: 20
finished Client ID:[26]: len: 20
finished Client ID:[27]: len: 20
finished Client ID:[28]: len: 20
finished Client ID:[29]: len: 20
finished Client ID:[

100%|██████████| 8/8 [00:03<00:00,  2.29it/s]


torch.Size([3, 512, 1024])


In [6]:
dataset_GTA = GTA5(root="/content/drive/MyDrive/data/data/GTA5",transform=train_transformations,fda=True,styles=styles)
train_dataloader = DataLoader(dataset_GTA, batch_size=BATCH_SIZE, shuffle=True, num_workers=2,drop_last=True)

In [7]:
test_dataset_B = Cityscapes('/content/drive/MyDrive/data/data/Cityscapes',transform=None,split='val')
test_dataset_A = Cityscapes('/content/drive/MyDrive/data/data/Cityscapes',transform=None,split='valA')

In [8]:
test_dataloader_B = DataLoader(test_dataset_B, batch_size=1, shuffle=False, num_workers=2,drop_last=True)
test_dataloader_A = DataLoader(test_dataset_A, batch_size=1, shuffle=False, num_workers=2,drop_last=True)

In [9]:
server = Server_FDA(clients,train_dataloader=train_dataloader,test_dataloaderA=test_dataloader_A,test_dataloaderB=test_dataloader_B,styles=styles)

server.train()



Starting epoch 121/600, LR = [5e-05]


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:381: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


00556
00201
24240
03766
22586
15542
24282
14934
17843
18073
14432
17847
03188
01950
02602


KeyboardInterrupt: ignored

In [ ]:
def _fast_hist(n_classes, label_true, label_pred):
      mask = (label_true >= 0) & (label_true < n_classes)
      hist = np.bincount(
          n_classes * label_true[mask].astype(int) + label_pred[mask],
          minlength=n_classes ** 2,
      ).reshape(n_classes, n_classes)
      return hist


def compute_mIoU(y_true,y_pred):
  
  y_pred = y_pred.cpu().numpy().flatten()
  y_true = y_true.cpu().numpy().flatten()
 
  hist = _fast_hist(19,y_true,y_pred)
  gt_sum = hist.sum(axis=1)
  mask = (gt_sum != 0)
  diag = np.diag(hist)
  iu = diag / (gt_sum + hist.sum(axis=0) - diag)
  mean_iu = np.mean(iu[mask])
  return mean_iu

In [ ]:
def evaluate(model,dataloader):
      net = model.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
      net.train(False) # Set Network to evaluation mode
      net=net.half()
      running_corrects = 0
      torch.cuda.empty_cache() 
      mIoU = 0
      count = 0

      for images, labels in tqdm(dataloader):
        images = images.half().to(DEVICE)
        labels = labels.half().to(DEVICE)
     
        outputs = net(images,test=True,use_test_resize=False)
        preds = outputs.argmax(dim=1)
        # print(outputs.size())
        mIoU += compute_mIoU(labels,preds)
        
        count += 1
      print("mIoU:",mIoU /count)
      return mIoU /count

In [ ]:
model = BiSeNetV2(n_classes=19,output_aux=True,pretrained=True)

In [ ]:
LEN = 32
def plotmIoU(train,label):
  epochs = np.array([a*5 for a in range(LEN)]).reshape(LEN,1)
  plt.figure()
  plt.ylim(0,max(train)+0.1)
  plt.plot(epochs,train,label=label)
  plt.legend()
  plt.show()

In [ ]:
train_mIoU_GTA = []
for i in range(0,160,10):
  print("checkpoint:",i)
  checkpoint = torch.load('/content/drive/MyDrive/magna/step4/checkpoint/long_esp_4.4_dataset_B_0.005/' +str(i)+'checkpoint.pt')
  model.load_state_dict(checkpoint['model_state_dict'],strict=False)
 
  train_mIoU_GTA.append(evaluate(model,test_dataloader_B))

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/magna/step4/checkpoint/long_esp_4.4_dataset_B_0.005/150checkpoint.pt')
test_mIoU_B = checkpoint['mIoUB']
test_mIoU_GTA = checkpoint['mIoUGTA']

In [ ]:
plotmIoU(test_mIoU_B[0:32],'test_mIoU_B')
plotmIoU(test_mIoU_GTA[0:32],'train_mIoU_GTA')
print(train_mIoU_GTA)